In [1]:
%pip install pyomo

# %apt-get install -y -qq glpk-utils

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.9 MB/s eta 0:00:00
Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 121752 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_5.0-1_amd64.deb ...
Unpacking libglpk40:amd64 (5.0-1) ...
Selecting previously unselected pac

In [6]:
import pyomo.environ as pyo

model = pyo.ConcreteModel()

# * index and param
model.i = pyo.Set(initialize=[1, 2, 3, 4, 5])
model.p = pyo.Param(model.i, initialize={1: 40_000, 2: 30_000, 3: 60_000, 4: 0, 5: 0})

# * D Variables
model.x = pyo.Var(model.i, domain=pyo.NonNegativeIntegers)
model.y = pyo.Var(model.i, domain=pyo.NonNegativeIntegers)
model.a = pyo.Var(model.i, domain=pyo.NonNegativeIntegers)
model.b = pyo.Var(model.i, domain=pyo.NonNegativeIntegers)
model.c = pyo.Var(model.i, domain=pyo.NonNegativeIntegers)
model.d = pyo.Var(model.i, domain=pyo.NonNegativeIntegers)

model.obj = pyo.Objective(
    expr=200 * sum((model.x[i] + model.y[i]) for i in model.i), sense=pyo.minimize
)


# * Constraints
def const1_rule(model, i):
    return model.a[i] <= model.x[i] * 40 * 4


model.const1 = pyo.Constraint(model.i, rule=const1_rule)


def const2_rule(model, i):
    return model.b[i] <= model.y[i] * 40 * 6


model.const2 = pyo.Constraint(model.i, rule=const2_rule)

# * Preprocess Constraints
model.const3 = pyo.Constraint(expr=model.p[1] == model.a[1] + model.c[1])
model.const4 = pyo.Constraint(expr=model.p[2] + model.c[1] == model.a[2] + model.c[2])
model.const5 = pyo.Constraint(expr=model.p[3] + model.c[2] == model.a[3] + model.c[3])
model.const6 = pyo.Constraint(expr=model.p[4] + model.c[3] == model.a[4] + model.c[4])
model.const7 = pyo.Constraint(expr=model.p[5] + model.c[4] == model.a[5] + model.c[5])
model.const8 = pyo.Constraint(expr=model.c[5] == 0)

# * Save Constraints
model.const9 = pyo.Constraint(expr=model.a[1] == model.b[1] + model.d[1])
model.const10 = pyo.Constraint(expr=model.a[2] + model.d[1] == model.b[2] + model.d[2])
model.const11 = pyo.Constraint(expr=model.a[3] + model.d[2] == model.b[3] + model.d[3])
model.const12 = pyo.Constraint(expr=model.a[4] + model.d[3] == model.b[4] + model.d[4])
model.const13 = pyo.Constraint(expr=model.a[5] + model.d[4] == model.b[5] + model.d[5])
model.const14 = pyo.Constraint(expr=model.d[5] == 0)

In [15]:
solver = pyo.SolverFactory("glpk", executable="/usr/bin/glpsol")

result = solver.solve(model)

print("Solver's status:", result.solver.status)

print("Solver's termination condition:", result.solver.termination_condition)

print("\nOptimal Solution:")
for i in model.i:
    print(f"x{i} =", model.x[i]())

print()

for i in model.i:
    print(f"y{i} =", model.y[i]())

print()
for i in model.i:
    print(f"a{i} =", model.a[i]())
print()
for i in model.i:
    print(f"b{i} =", model.b[i]())
print()
for i in model.i:
    print(f"c{i} =", model.c[i]())
print()
for i in model.i:
    print(f"d{i} =", model.d[i]())
print()
print("Objective function's value (z) =", model.obj())

Solver's status: ok
Solver's termination condition: optimal

Optimal Solution:
x1 = 0.0
x2 = 0.0
x3 = 0.0
x4 = 0.0
x5 = 813.0

y1 = 0.0
y2 = 0.0
y3 = 0.0
y4 = 0.0
y5 = 542.0

a1 = 0.0
a2 = 0.0
a3 = 0.0
a4 = 0.0
a5 = 130000.0

b1 = 0.0
b2 = 0.0
b3 = 0.0
b4 = 0.0
b5 = 130000.0

c1 = 40000.0
c2 = 70000.0
c3 = 130000.0
c4 = 130000.0
c5 = 0.0

d1 = 0.0
d2 = 0.0
d3 = 0.0
d4 = 0.0
d5 = 0.0

Objective function's value (z) = 271000.0
